# Erase Old Catalog
This notebook drops all user-defined functions, tables, and views from a catalog and then removes the schemas. Provide the catalog name with the widget below and run all cells.

In [ ]:

dbutils.widgets.text("1.catalog", "old_catalog")

catalog = dbutils.widgets.get("1.catalog")


In [ ]:

skip_schemas = {"information_schema", "pg_catalog"}

schemas_df = spark.sql(f"SHOW SCHEMAS IN `{catalog}`")

for row in schemas_df.collect():
    schema = row[0]
    if schema.lower() in skip_schemas:
        continue

    # Drop functions first
    funcs_df = spark.sql(f"SHOW USER FUNCTIONS IN `{catalog}`.`{schema}`")
    for f in funcs_df.collect():
        func_name = f[0]
        spark.sql(f"DROP FUNCTION IF EXISTS `{catalog}`.`{schema}`.`{func_name}`")

    # Drop tables and views
    tables_df = spark.sql(f"SHOW TABLES IN `{catalog}`.`{schema}`")
    for t in tables_df.collect():
        table_name = t.tableName
        if t.tableType == 'VIEW':
            spark.sql(f"DROP VIEW IF EXISTS `{catalog}`.`{schema}`.`{table_name}`")
        else:
            spark.sql(f"DROP TABLE IF EXISTS `{catalog}`.`{schema}`.`{table_name}`")

    # Drop the schema itself
    spark.sql(f"DROP SCHEMA IF EXISTS `{catalog}`.`{schema}`")


In [ ]:
print(f'Catalog `{catalog}` cleanup complete.')